In [1]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git
!pip install datasets -q
!pip install peft -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.7 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
import requests
import torch

In [4]:
model_id = "google/paligemma-3b-mix-224"

In [6]:
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/paligemma-3b-mix-224.
401 Client Error. (Request ID: Root=1-66453fcb-724f52ed7b892299700c7e82;028bd1be-8e5f-471d-b418-b57d406885d3)

Cannot access gated repo for url https://huggingface.co/google/paligemma-3b-mix-224/resolve/main/config.json.
Access to model google/paligemma-3b-mix-224 is restricted. You must be authenticated to access it.

In [ ]:
prompt = "What car is this?"

In [ ]:
image_file = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"

In [ ]:
raw_image = Image.open(requests.get(image_file, stream=True).raw)

In [ ]:
inputs = processor(prompt, raw_image, return_tensors="pt")

In [ ]:
output = model.generate(**inputs, max_new_tokens=20)

In [ ]:
output

In [ ]:
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map= {"": 0}
)

# FT

In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset("HuggingFaceM4/VQAv2", split="train")

In [ ]:
cols_remove = ["question_type", "answers", "answer_type", "question_id"]
ds = ds.remove_columns(cols_remove)
ds = ds.train_test_split(test_size=0.1)
train_ds = ds["train"]
test_ds = ds["test"]

In [ ]:
from transformers import PaliGemmaProcessor

In [ ]:
model_id = "google/paligemma-3b-pt-224"
processor = PaliGemmaProcessor(model_id)

In [ ]:
device = "cuda"

In [ ]:
image_token = processor.tokenizer.convert_tokens_to_ids("<image>")

In [ ]:
def collate_fn(examples):
    texts = ["answer " + example["question"] + "\n" + example['multiple_choice_answer'] for example in examples]
    images = [example["image"].convert("RGB") for example in examples]
    tokens = processor(
        text=texts, images=images,
        return_tensors="pt", padding="longest",
        tokenize_newline_separately=False
    )
    labels = tokens["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token] = -100
    tokens["labels"] = labels
    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens

In [ ]:
from peft import get_peft_model, LoraConfig

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map= {"":0}
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments

In [ ]:
args=TrainingArguments(
    num_train_epochs=2,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    learning_rate=2e-5,
    weight_decay=1e-6,
    adam_beta2=0.999,
    logging_steps=100,
    optim="adamw_hf",
    save_strategy="steps",
    save_steps=1000,
    push_to_hub=True,
    save_total_limit=1,
    bf16=True,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn
)

In [ ]:
trainer.train()